# Neo4j와 LangChain을 활용한 영화 추천 시스템

---

## 1. Neo4J DB 환경 설정

In [1]:
import os
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv(override=True)

True

In [2]:
from langchain_neo4j import Neo4jGraph

# LangChain 도구 활용 - DB 연결 객체 초기화 
graph = Neo4jGraph( 
    url=os.getenv("NEO4J_URI"), 
    username=os.getenv("NEO4J_USERNAME"), 
    password=os.getenv("NEO4J_PASSWORD"),
    database=os.getenv("NEO4J_DATABASE")
)

In [3]:
# 테스트 쿼리 실행 
cypher_query = """
MATCH (n:Movie)
RETURN COUNT(n) AS Movie_Count
"""

graph.query(cypher_query)

[{'Movie_Count': 4803}]

---

## 2. Neo4j Cypher 쿼리 분석 - 영화 데이터베이스 활용


1. **그래프 패턴 매칭**:
   - Cypher의 가장 큰 강점은 노드-관계 패턴을 직관적으로 표현할 수 있다는 점입니다.
   - 화살표(`->`), 관계 타입(`[:ACTED_IN]`) 등을 통해 그래프 구조를 명확히 표현합니다.

2. **데이터 집계 및 변환**:
   - `count()`, `collect()` 등의 함수를 사용해 결과를 집계하고 변환합니다.
   - `WITH` 절을 사용해 중간 결과를 다음 단계로 전달합니다.

3. **가중치 기반 점수 계산**:
   - 복합 추천 시스템에서는 다양한 요소(장르, 배우)에 가중치를 부여하여 추천 점수를 계산합니다.
   - `size()` 함수로 배열 크기를 계산하고 이를 곱셈 연산에 활용합니다.

4. **매개변수 사용**:
   - `$actor_name`, `$movie_title`과 같은 매개변수를 사용해 동적 쿼리를 구성합니다.
   - 이는 SQL의 준비된 문장(Prepared Statement)과 유사한 역할을 합니다.

5. **OPTIONAL MATCH**:
   - SQL의 LEFT JOIN과 유사한 개념으로, 일치하는 패턴이 없어도 결과를 반환합니다.
   - 복합 추천 시스템에서 장르나 배우가 일치하지 않는 경우에도 계속 처리할 수 있게 합니다.
   

### 2.1 기본 분석 쿼리

- **평점 높은 영화 상위 10개 조회**:

    - **MATCH (m:Movie)**: Movie 라벨을 가진 모든 노드를 찾습니다.
    - **WHERE m.rating IS NOT NULL**: 평점이 NULL이 아닌 영화만 필터링합니다.
    - **RETURN**: 영화 제목, 개봉일, 평점을 반환하고 의미 있는 열 이름으로 별칭을 지정합니다.
    - **ORDER BY m.rating DESC**: 평점 기준 내림차순 정렬합니다.
    - **LIMIT 10**: 상위 10개 결과만 반환합니다.

In [4]:
# 평점 기준 상위 10개 영화를 조회하는 Cypher 쿼리
cypher_query = """
MATCH (m:Movie)                                 // Movie 라벨을 가진 모든 노드 매칭
WHERE m.rating IS NOT NULL                      // 평점 값이 존재하는 영화만 필터링
RETURN m.title AS Movie,                        // 영화 제목을 'Movie'라는 별칭으로 반환
       m.released AS Released,                  // 개봉일을 'Released'라는 별칭으로 반환
       m.rating AS Rating                       // 평점을 'Rating'이라는 별칭으로 반환
ORDER BY m.rating DESC                          // 평점 기준 내림차순 정렬 (높은 평점부터)
LIMIT 10                                        // 상위 10개 결과만 반환
"""

# Neo4j 데이터베이스에 쿼리 실행 및 결과 반환
result = graph.query(cypher_query)

result

[{'Movie': 'Little Big Top', 'Released': '2006-01-01', 'Rating': 10.0},
 {'Movie': 'Me You and Five Bucks', 'Released': '2015-07-07', 'Rating': 10.0},
 {'Movie': 'Sardaarji', 'Released': '2015-06-26', 'Rating': 9.5},
 {'Movie': 'Counting', 'Released': '2015-02-09', 'Rating': 8.3},
 {'Movie': 'Whiplash', 'Released': '2014-10-10', 'Rating': 8.3},
 {'Movie': 'The Dark Knight', 'Released': '2008-07-16', 'Rating': 8.2},
 {'Movie': 'Guten Tag, Ramón', 'Released': '2013-10-18', 'Rating': 8.1},
 {'Movie': 'Inception', 'Released': '2010-07-14', 'Rating': 8.1},
 {'Movie': 'Interstellar', 'Released': '2014-11-05', 'Rating': 8.1},
 {'Movie': 'Room', 'Released': '2015-10-16', 'Rating': 8.1}]

In [5]:
# 데이터프레임 변환
import pandas as pd

pd.DataFrame(result)

,Movie,Released,Rating
0,Little Big Top,2006-01-01,10.0
1,Me You and Five Bucks,2015-07-07,10.0
2,Sardaarji,2015-06-26,9.5
3,Counting,2015-02-09,8.3
4,Whiplash,2014-10-10,8.3
5,The Dark Knight,2008-07-16,8.2
6,"Guten Tag, Ramón",2013-10-18,8.1
7,Inception,2010-07-14,8.1
8,Interstellar,2014-11-05,8.1
9,Room,2015-10-16,8.1


- **출연 영화 많은 배우 상위 10명**:

    - **MATCH (p:Person)-[:ACTED_IN]->(m:Movie)**: Person 노드에서 ACTED_IN 관계를 통해 Movie 노드로 연결된 패턴을 찾습니다.
    - **count(m)**: 배우별로 연결된 영화 노드 수를 계산합니다.
    - **ORDER BY MovieCount DESC**: 출연 영화 수 기준 내림차순 정렬합니다.

In [ ]:
# 출연 영화가 많은 배우 상위 10명을 조회하는 Cypher 쿼리
cypher_query = """
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)         // Person 노드(배우)에서 ACTED_IN 관계를 통해 Movie 노드로 연결된 패턴 찾기
                                               // p 변수는 Person 타입의 노드를 참조하고, m 변수는 Movie 타입의 노드를 참조함
RETURN p.name AS Actor,                         // 배우 이름(p.name 속성)을 'Actor'라는 별칭으로 결과에 포함
       count(m) AS MovieCount                   // 각 배우별 출연한 영화 수를 집계 함수 count()로 계산하여 'MovieCount'라는 별칭으로 반환
                                               // count(m)은 각 배우(p)가 ACTED_IN 관계로 연결된 영화(m) 노드의 개수를 계산함
ORDER BY MovieCount DESC                        // 출연 영화 수(MovieCount) 기준 내림차순(DESC) 정렬
                                               // 가장 많은 영화에 출연한 배우가 결과의 상단에 위치하게 됨
LIMIT 10                                        // 정렬된 결과 중 상위 10개의 레코드만 반환
                                               // 즉, 가장 많은 영화에 출연한 상위 10명의 배우만 결과에 포함됨
"""
                                                                                                                                                                                             
# Neo4j 데이터베이스에 쿼리 실행 및 결과 반환
graph.query(cypher_query)

[{'Actor': 'Samuel L. Jackson', 'MovieCount': 21},
 {'Actor': 'Woody Harrelson', 'MovieCount': 21},
 {'Actor': 'Will Ferrell', 'MovieCount': 21},
 {'Actor': 'Paul Rudd', 'MovieCount': 20},
 {'Actor': 'Robert De Niro', 'MovieCount': 20},
 {'Actor': 'Mark Wahlberg', 'MovieCount': 20},
 {'Actor': 'Scarlett Johansson', 'MovieCount': 20},
 {'Actor': 'Matt Damon', 'MovieCount': 20},
 {'Actor': 'Justin Long', 'MovieCount': 19},
 {'Actor': 'Gerard Butler', 'MovieCount': 19}]

- **장르별 영화 수**:

    - **MATCH (m:Movie)-[:IN_GENRE]->(g:Genre)**: Movie 노드에서 IN_GENRE 관계를 통해 Genre 노드로 연결된 패턴을 찾습니다.
    - **count(m)**: 각 장르별로 연결된 영화 수를 계산합니다.
    - **ORDER BY MovieCount DESC**: 영화 수 기준 내림차순 정렬합니다.

In [7]:
# 장르별 영화 수를 조회하는 Cypher 쿼리
cypher_query = """
MATCH (m:Movie)-[:IN_GENRE]->(g:Genre)          // Movie 노드에서 IN_GENRE 관계를 통해 Genre 노드로 연결된 패턴 찾기
                                               // m 변수는 Movie 타입의 노드를 참조하고, g 변수는 Genre 타입의 노드를 참조함
                                               // IN_GENRE 관계는 영화가 어떤 장르에 속하는지를 나타내는 관계임
RETURN g.name AS Genre,                         // 장르 이름(g.name 속성)을 'Genre'라는 별칭으로 결과에 포함
                                               // g.name은 장르 노드의 name 속성으로, 장르의 이름을 나타냄
       count(m) AS MovieCount                   // 각 장르별 영화 수를 집계 함수 count()로 계산하여 'MovieCount'라는 별칭으로 반환
                                               // count(m)은 각 장르(g)에 IN_GENRE 관계로 연결된 영화(m) 노드의 개수를 계산함
ORDER BY MovieCount DESC                        // 영화 수(MovieCount) 기준 내림차순(DESC) 정렬
                                               // 가장 많은 영화가 속한 장르가 결과의 상단에 위치하게 됨
"""

# Neo4j 데이터베이스에 쿼리 실행 및 결과 반환
graph.query(cypher_query)

[{'Genre': 'Drama', 'MovieCount': 1219},
 {'Genre': 'Comedy', 'MovieCount': 879},
 {'Genre': 'Thriller', 'MovieCount': 673},
 {'Genre': 'Action', 'MovieCount': 584},
 {'Genre': 'Romance', 'MovieCount': 422},
 {'Genre': 'Adventure', 'MovieCount': 387},
 {'Genre': 'Crime', 'MovieCount': 337},
 {'Genre': 'Horror', 'MovieCount': 306},
 {'Genre': 'Family', 'MovieCount': 289},
 {'Genre': 'Science Fiction', 'MovieCount': 263},
 {'Genre': 'Fantasy', 'MovieCount': 221},
 {'Genre': 'Mystery', 'MovieCount': 175},
 {'Genre': 'Animation', 'MovieCount': 153},
 {'Genre': 'History', 'MovieCount': 99},
 {'Genre': 'Music', 'MovieCount': 89},
 {'Genre': 'Documentary', 'MovieCount': 87},
 {'Genre': 'War', 'MovieCount': 64},
 {'Genre': 'Western', 'MovieCount': 32},
 {'Genre': 'Foreign', 'MovieCount': 17},
 {'Genre': 'TV Movie', 'MovieCount': 7}]

### 2.2 관계 기반 고급 분석

- **특정 배우와 함께 출연한 배우들 찾기**:

    - **MATCH (actor:Person {name: $actor_name})**: 지정된 이름을 가진 배우를 찾습니다.
    - **-[:ACTED_IN]->(m:Movie)<-[:ACTED_IN]-(co_actor:Person)**: 해당 배우가 출연한 영화에 함께 출연한 다른 배우들을 찾습니다.
    - **WHERE actor <> co_actor**: 동일 배우는 제외합니다(자기 자신 제외).
    - **count(m)**: 함께 출연한 영화 수를 계산합니다.
    - **params={"actor_name": "Tom Hanks"}**: 쿼리 매개변수로 특정 배우 이름을 전달합니다.

In [8]:
cypher_query = """
MATCH (actor:Person {name: $actor_name})-[:ACTED_IN]->(m:Movie)<-[:ACTED_IN]-(co_actor:Person)  // 특정 배우(actor)와 함께 출연한 다른 배우(co_actor)를 찾는 패턴
                                                                                               // actor는 name 속성이 $actor_name 파라미터와 일치하는 Person 노드
                                                                                               // -[:ACTED_IN]-> 관계는 배우가 영화에 출연했음을 나타냄
                                                                                               // <-[:ACTED_IN]- 관계는 다른 배우(co_actor)도 같은 영화에 출연했음을 나타냄
WHERE actor <> co_actor                                                                        // 자기 자신은 결과에서 제외 (actor와 co_actor가 같지 않은 경우만 포함)
RETURN co_actor.name AS CoActor, count(m) AS MoviesTogether                                    // 함께 출연한 배우 이름과 함께 출연한 영화 수를 반환
                                                                                               // co_actor.name은 함께 출연한 배우의 이름
                                                                                               // count(m)은 두 배우가 함께 출연한 영화의 수를 계산
ORDER BY MoviesTogether DESC                                                                   // 함께 출연한 영화 수를 기준으로 내림차순 정렬 (가장 많이 함께 출연한 배우가 상위에 표시)
LIMIT 5                                                                                        // 상위 5명의 배우만 결과에 포함
"""

# 쿼리 실행 시 "Tom Hanks"를 $actor_name 파라미터 값으로 전달
graph.query(cypher_query, params={"actor_name": "Tom Hanks"})                                  

[{'CoActor': 'Julia Roberts', 'MoviesTogether': 2},
 {'CoActor': 'Audrey Tautou', 'MoviesTogether': 1},
 {'CoActor': 'Ian McKellen', 'MoviesTogether': 1},
 {'CoActor': 'Martin Sheen', 'MoviesTogether': 1},
 {'CoActor': 'Paul Bettany', 'MoviesTogether': 1}]

- **배우-감독 협업 관계 분석**:

    - **MATCH (a:Person)-[:ACTED_IN]->(m:Movie)<-[:DIRECTED]-(d:Person)**: 배우가 출연하고 감독이 연출한 영화를 찾는 패턴입니다.
    - **WHERE a.name = $actor_name**: 특정 배우로 필터링합니다.
    - **count(m)**: 배우와 감독이 함께 작업한 영화 수를 계산합니다.
    - **params={"actor_name": "Leonardo DiCaprio"}**: 쿼리 매개변수로 배우 이름을 전달합니다.

In [9]:
cypher_query = """
MATCH (a:Person)-[:ACTED_IN]->(m:Movie)<-[:DIRECTED]-(d:Person)  // 배우(a)가 출연한 영화(m)와 그 영화를 감독한 감독(d)을 찾는 패턴
                                                                 // (a:Person): 배우 노드
                                                                 // -[:ACTED_IN]->: 배우가 영화에 출연했다는 관계
                                                                 // (m:Movie): 영화 노드
                                                                 // <-[:DIRECTED]-: 감독이 영화를 감독했다는 관계
                                                                 // (d:Person): 감독 노드
WHERE a.name = $actor_name                                       // 특정 배우 이름으로 필터링 (파라미터로 전달된 배우 이름과 일치하는 경우만 선택)
RETURN d.name AS Director, count(m) AS CollaborationCount        // 감독 이름과 해당 감독과 함께 작업한 영화 수를 반환
                                                                 // d.name: 감독의 이름
                                                                 // count(m): 배우와 감독이 함께 작업한 영화의 수
ORDER BY CollaborationCount DESC                                 // 협업 횟수를 기준으로 내림차순 정렬 (가장 많이 함께 작업한 감독이 상위에 표시)
LIMIT 5                                                          // 상위 5개 결과만 반환  
"""

# 쿼리 실행 시 "Leonardo DiCaprio"를 $actor_name 파라미터 값으로 전달
graph.query(cypher_query, params={"actor_name": "Leonardo DiCaprio"})  

[{'Director': 'Martin Scorsese', 'CollaborationCount': 3},
 {'Director': 'Ridley Scott', 'CollaborationCount': 1},
 {'Director': 'Sam Mendes', 'CollaborationCount': 1},
 {'Director': 'Christopher Nolan', 'CollaborationCount': 1},
 {'Director': 'Edward Zwick', 'CollaborationCount': 1}]

### 2.3 그래프 기반 추천 시스템

- **장르 기반 영화 추천**:

    - **MATCH (m:Movie {title: $movie_title})**: 지정된 제목의 영화를 찾습니다.
    - **-[:IN_GENRE]->(g:Genre)<-[:IN_GENRE]-(rec:Movie)**: 같은 장르를 공유하는 다른 영화를 찾습니다.
    - **WHERE m <> rec AND rec.rating > 7.0**: 원본 영화 자체는 제외하고, 평점이 7.0 이상인 영화만 필터링합니다.
    - **collect(g.name) AS SharedGenres**: 공유하는 모든 장르를 배열로 수집합니다.
    - **ORDER BY Rating DESC, size(SharedGenres) DESC**: 평점이 높은 순서, 그리고 공유 장르 수가 많은 순서로 정렬합니다.

In [10]:
cypher_query = """
MATCH (m:Movie {title: $movie_title})-[:IN_GENRE]->(g:Genre)<-[:IN_GENRE]-(rec:Movie)  // 입력된 영화(m)와 같은 장르(g)에 속한 다른 영화(rec)를 찾는 패턴
                                                                                       // (m:Movie {title: $movie_title}): 파라미터로 전달된 제목을 가진 영화 노드
                                                                                       // -[:IN_GENRE]->: 영화가 특정 장르에 속한다는 관계
                                                                                       // (g:Genre): 장르 노드
                                                                                       // <-[:IN_GENRE]-: 다른 영화가 같은 장르에 속한다는 관계
                                                                                       // (rec:Movie): 추천할 다른 영화 노드
WHERE m <> rec AND rec.rating > 7.0                                                    // 조건: 원본 영화와 추천 영화는 서로 다른 영화여야 하고(m <> rec),
                                                                                       // 추천 영화의 평점은 7.0 초과여야 함(rec.rating > 7.0)
RETURN rec.title AS RecommendedMovie,                                                  // 추천 영화의 제목을 RecommendedMovie로 반환
       rec.rating AS Rating,                                                           // 추천 영화의 평점을 Rating으로 반환
       collect(g.name) AS SharedGenres                                                 // 공유하는 모든 장르 이름을 배열로 모아 SharedGenres로 반환
                                                                                       // collect(): 여러 행의 값을 하나의 배열로 수집하는 집계 함수
ORDER BY Rating DESC, size(SharedGenres) DESC                                          // 정렬 기준: 1) 평점 내림차순(높은 평점 우선)
                                                                                       // 2) 공유 장르 수 내림차순(더 많은 장르를 공유할수록 우선)
                                                                                       // size(): 배열의 크기를 반환하는 함수
LIMIT 5                                                                                // 상위 5개 결과만 반환
"""

# 쿼리 실행 시 "Apollo 13"을 $movie_title 파라미터 값으로 전달
graph.query(cypher_query, params={"movie_title": "Apollo 13"})                         

[]

- **복합 추천 (장르+배우 가중치)**:

    - **MATCH (m:Movie {title: $movie_title})**: 지정된 영화를 시작점으로 합니다.
    - **MATCH (rec:Movie) WHERE m <> rec AND rec.rating > 7.0**: 다른 모든 영화 중 평점 7.0 이상인 것만 고려합니다.
    - **OPTIONAL MATCH (m)-[:IN_GENRE]->(g:Genre)<-[:IN_GENRE]-(rec)**: 공통 장르를 찾습니다(있을 수도 없을 수도 있음).
    - **OPTIONAL MATCH (m)<-[:ACTED_IN]-(a:Person)-[:ACTED_IN]->(rec)**: 공통 배우를 찾습니다(있을 수도 없을 수도 있음).
    - **size(genres) * 2 + size(actors) * 3 AS score**: 추천 점수를 계산합니다. 장르는 가중치 2, 배우는 가중치 3을 부여합니다.
    - **WHERE score > 0**: 최소한 하나 이상의 공통점이 있는 영화만 추천합니다.
    - **ORDER BY score DESC, Rating DESC**: 추천 점수가 높은 순서, 그리고 평점이 높은 순서로 정렬합니다.


In [13]:
cypher_query = """
MATCH (m:Movie {title: $movie_title})                                                  // 파라미터로 전달된 제목을 가진 영화 노드를 찾음
MATCH (rec:Movie) WHERE m <> rec AND rec.rating > 7.0                                  // 원본 영화와 다르고 평점이 7.0 초과인 모든 영화를 찾음
OPTIONAL MATCH (m)-[:IN_GENRE]->(g:Genre)<-[:IN_GENRE]-(rec)                           // 원본 영화와 추천 영화가 공유하는 장르를 찾음 (없을 수도 있음)

WITH m, rec, COLLECT(g.name) AS genres                                                 // 공유 장르 이름을 배열로 수집하여 다음 단계로 전달
OPTIONAL MATCH (m)<-[:ACTED_IN]-(a:Person)-[:ACTED_IN]->(rec)                          // 원본 영화와 추천 영화에 모두 출연한 배우를 찾음 (없을 수도 있음)

WITH m, rec, genres, COLLECT(a.name) AS actors                                         // 공유 배우 이름을 배열로 수집하여 다음 단계로 전달

WITH rec, 
     size(genres) * 2 + size(actors) * 3 AS score,                                     // 추천 점수 계산: 공유 장르 수 x 2 + 공유 배우 수 x 3
     genres, 
     actors
WHERE score > 0                                                                        // 최소한 하나 이상의 공통점이 있는 영화만 필터링 (점수가 0보다 큰 경우)
RETURN rec.title AS RecommendedMovie,                                                  // 추천 영화의 제목 반환
       rec.rating AS Rating,                                                           // 추천 영화의 평점 반환
       score AS RecommendationScore,                                                   // 계산된 추천 점수 반환
       genres AS SharedGenres,                                                         // 공유하는 장르 목록 반환
       actors AS SharedActors                                                          // 공유하는 배우 목록 반환
       
ORDER BY score DESC, Rating DESC                                                       // 1) 추천 점수 내림차순, 2) 평점 내림차순으로 정렬
LIMIT 5                                                                                // 상위 5개 결과만 반환
"""

# 쿼리 실행 시 "Hide and Seek"을 $movie_title 파라미터 값으로 전달
graph.query(cypher_query, params={"movie_title": "Hide and Seek"})                         

[{'RecommendedMovie': 'Taken',
  'Rating': 7.2,
  'RecommendationScore': 5,
  'SharedGenres': ['Thriller'],
  'SharedActors': ['Famke Janssen']},
 {'RecommendedMovie': 'Inception',
  'Rating': 8.1,
  'RecommendationScore': 4,
  'SharedGenres': ['Thriller', 'Mystery'],
  'SharedActors': []},
 {'RecommendedMovie': 'The Prestige',
  'Rating': 8.0,
  'RecommendationScore': 4,
  'SharedGenres': ['Thriller', 'Mystery'],
  'SharedActors': []},
 {'RecommendedMovie': 'Gone Girl',
  'Rating': 7.9,
  'RecommendationScore': 4,
  'SharedGenres': ['Thriller', 'Mystery'],
  'SharedActors': []},
 {'RecommendedMovie': 'Shutter Island',
  'Rating': 7.8,
  'RecommendationScore': 4,
  'SharedGenres': ['Thriller', 'Mystery'],
  'SharedActors': []}]